In [ ]:
attribute_retrieval_questions = [
    "How many total sessions are there at re:Invent?", 
    "List all sessions for developers.", 
    "How many GenAI-related sessions are there at this re:Invent?", 
    "What are the session codes related to API Gateway?", 
    "List all sessions targeted at data scientists.", 
    "How many sessions focus on AWS Lambda?", 
    "What is the number of sessions about cloud security?", 
    "List all sessions that mention Amazon S3." 
]

contextual_exploration_questions = [ 
    "Can you recommend sessions where I can learn about fine-tuning generative AI models?", 
    "Which sessions would be most beneficial for someone new to cloud computing?", 
    "What are the most relevant sessions about tools for Kubernetes?", 
    "Are there any sessions that explain the process of database migration?", 
    "Can you suggest sessions that focus on improving application performance?", 
    "What are the key sessions discussing serverless architecture?", 
    "Can you recommend sessions that cover best practices in DevOps?", 
    "What sessions might help me understand the integration of AI in cloud services?" 
]

In [ ]:
import yaml
from string import Template

def load_prompt(func_name, prompt_file='prompts.yaml'):
    with open(prompt_file, 'r') as file:
        prompts = yaml.safe_load(file)
    
    if func_name not in prompts:
        raise KeyError(f"Function '{func_name}' not found in the prompt file.")
    
    func_prompts = prompts[func_name]
        
    if 'sys_template' in func_prompts and 'user_template' in func_prompts:
        return func_prompts['sys_template'], func_prompts['user_template']
    elif 'system_prompt' in func_prompts and 'user_prompt' in func_prompts:
        return func_prompts['system_prompt'], func_prompts['user_prompt']
    else:
        raise KeyError(f"'sys_template' or 'system_prompt' not found for '{func_name}'")

In [ ]:
import boto3
from botocore.config import Config

region = 'us-west-2'
def init_bedrock_client(region):
    retry_config = Config(
        region_name=region,
        retries={"max_attempts": 10, "mode": "standard"}
    )
    return boto3.client("bedrock-runtime", region_name=region, config=retry_config)

def converse_with_bedrock(model_id, sys_prompt, usr_prompt):
    temperature = 0.5
    top_p = 0.9
    inference_config = {"temperature": temperature, "topP": top_p}
    response = boto3_client.converse(
        modelId=model_id,
        messages=usr_prompt, 
        system=sys_prompt,
        inferenceConfig=inference_config,
    )
    return response

def create_prompt(sys_template, user_template, **kwargs):
    sys_prompt = [{"text": sys_template.format(**kwargs)}]
    usr_prompt = [{"role": "user", "content": [{"text": user_template.format(**kwargs)}]}]
    return sys_prompt, usr_prompt

def create_prompt_without_parameter(sys_template, user_template):
    sys_prompt = [{"text": sys_template}]
    usr_prompt = [{"role": "user", "content": [{"text": user_template}]}]
    return sys_prompt, usr_prompt

boto3_client = init_bedrock_client(region)

In [ ]:
topics = ['AI/ML', 'Analytics', 'Architecture', 'Cloud Operations', 'Compute', 'Serverless & Containers', 'Database', 'Developer Tools', 'Security', 'Storage', 'Migration & Modernization', 'IoT', 'Other']
audience_types = ['Developers', 'System Administrator', 'IT Administrator', 'Data Scientists', 'Security Professionals', 'Other']
session_format = ['Breakout Session', 'Chalk Talk', 'Builder session', 'Workshop', 'Lightening Talk']

In [ ]:
def search_tool_selection(question):
    sys_template, user_template = load_prompt('search_tool_selection')
    model_id="anthropic.claude-3-haiku-20240307-v1:0"
    sys_prompt, usr_prompt = create_prompt(sys_template, user_template, question=question)
    response = converse_with_bedrock(model_id, sys_prompt, usr_prompt)
    return response['output']['message']['content'][0]['text']

In [ ]:
for question in attribute_retrieval_questions:
    response = search_tool_selection(question)
    print(response)


In [ ]:
for question in contextual_exploration_questions:
    response = search_tool_selection(question)
    print(response)

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection
from dotenv import load_dotenv
import os

load_dotenv()
host = os.getenv('OPENSEARCH_HOST')
user = os.getenv('OPENSEARCH_USER')
password = os.getenv('OPENSEARCH_PASSWORD')
region = 'us-east-1'
index_name = 'reinvent_session'

os_client = OpenSearch(
    hosts = [{'host': host.replace("https://", ""), 'port': 443}],
    http_auth = (user, password),
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

In [ ]:
def search_by_text(topics, audience_types, session_format, question):
    sys_template, user_template = load_prompt('search_by_text')
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0"
    #model_id="anthropic.claude-3-sonnet-20240229-v1:0"
    #model_id="anthropic.claude-3-haiku-20240307-v1:0"
    sys_prompt, user_prompt = create_prompt(sys_template, user_template, question=question, topics=topics, audience_types=audience_types, session_format=session_format)
    response = converse_with_bedrock(model_id, sys_prompt, user_prompt)
    return response['output']['message']['content'][0]['text'], sys_prompt, user_prompt

question = "How many total sessions are there at re:Invent?"
search_query, previous_sys_prompt, previous_user_prompt = search_by_text(topics, audience_types, session_format, question)
print("search_query:", search_query)

In [ ]:
def execute_query(os_client, index_name, search_query):
    try:
        search_result = os_client.search(
            index=index_name,
            body=search_query
        )
        return search_result, None
    except Exception as e:
        return None, str(e)

In [ ]:
def search_by_text_as_fallback(previous_sys_prompt, previous_user_prompt, failed_query, error_message):
    sys_prompt, user_prompt = load_prompt('search_by_text_as_fallback')
    
    fallback_sys_prompt = sys_prompt.format(
        previous_sys_prompt=previous_sys_prompt,
        previous_user_prompt=previous_user_prompt,
        failed_query=failed_query,
        error_message=error_message
    )

    fallback_user_prompt = user_prompt

    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
    sys_prompt, usr_prompt = create_prompt_without_parameter(fallback_sys_prompt, fallback_user_prompt)
    response = converse_with_bedrock(model_id, sys_prompt, usr_prompt)

    return response['output']['message']['content'][0]['text']

### Test Case
failed_query = """{
  "size": 0,
  "aggres": {
    "total_sessions": {
      "value_count": {
        "field": "code"
      }
      }
    }
  }
}"""

search_result, error = execute_query(os_client, index_name, failed_query)
print(error)
revised_query = search_by_text_as_fallback(previous_sys_prompt, previous_user_prompt, failed_query, error)
print(revised_query)


In [ ]:
def search_by_text_as_fallback(previous_sys_prompt, previous_user_prompt, failed_query, error_message):
    fallback_sys_prompt = f"""
    The previous attempt to create a query failed. Here are the details:

    System Prompt used:
    ```
    {previous_sys_prompt}
    ```

    User Prompt used:
    ```
    {previous_user_prompt}
    ```

    The query generated from the above prompts:
    {failed_query}

    Error messages for failure:
    {error_message}
    """

    fallback_user_prompt = """
    Based on this information, please revise the query to fix this error and try again. 
    Provide only the revised DSL query without any additional explanation.
    """

    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0"
    sys_prompt, user_prompt = create_prompt_without_parameter(fallback_sys_prompt, fallback_user_prompt)
    response = converse_with_bedrock(model_id, sys_prompt, user_prompt)

    return response['output']['message']['content'][0]['text']
    

### Test Case
failed_query = """{
  "size": 0,
  "aggres": {
    "total_sessions": {
      "value_count": {
        "field": "code"
      }
      }
    }
  }
}"""

search_result, error = execute_query(os_client, index_name, failed_query)
print(error)
revised_query = search_by_text_as_fallback(previous_sys_prompt, previous_user_prompt, failed_query, error)
print(revised_query)


In [ ]:
try:
    search_result, error = execute_query(os_client, index_name, search_query)

    if error:
        print(f"attempt failed. Error: {error}")
        print(f"Attepting fallback")
        revised_query = search_by_text_as_fallback(previous_sys_prompt, previous_user_prompt, search_query, error)
        search_result, error = execute_query(os_client, index_name, revised_query)

        if error:
            raise Exception("Both initial and fallback queries failed. Final error: {error}")
    
except Exception as e:
    print(f"An error occurred: {str(e)}")
    exit(1)

print(search_result)

In [ ]:
import json

def generate_answer_with_text_search(search_result, search_query, question):
    sys_prompt, user_prompt = load_prompt('generate_answer_with_text_search')
    
    search_query_str = json.dumps(search_query, indent=2)
    search_result_str = json.dumps(search_result, indent=2)

    user_prompt = user_prompt.format(
        question=question,
        search_query=search_query_str,
        search_result=search_result_str
    )

    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
    sys_prompt, user_prompt = create_prompt_without_parameter(sys_prompt, user_prompt)
    response = converse_with_bedrock(model_id, sys_prompt, user_prompt)

    return response['output']['message']['content'][0]['text']

answer = generate_answer_with_text_search(search_result, search_query, question)
print(answer)

In [ ]:
def format_search_results(search_result, max_results=20):
    hits = search_result.get('hits', {}).get('hits', [])

    if hits:
        total_hits = search_result.get('hits', {}).get('total', {}).get('value', 0)
        formatted_results = []

        for hit in hits[:max_results]:
            source = hit.get('_source', {})
            code = source.get('code', 'N/A')
            title = source.get('title', 'N/A')
            formatted_results.append(f"{code}: {title}")

        result_str = "\n".join(formatted_results)

        if total_hits > max_results:
            result_str += f"\n\n... (Showing {max_results} out of {total_hits} results)"

        return result_str
    else:
        return json.dumps(search_result, indent=2)

In [ ]:
for question in attribute_retrieval_questions:
    response = search_tool_selection(question)
    
    print(f"\n\nquestion: {question}")
    if response == "search_by_text":
        search_query, previous_sys_prompt, previous_user_prompt = search_by_text(topics, audience_types, session_format, question) 
        search_result, error = execute_query(os_client, index_name, search_query)
        if error:
            print(f"attempt failed. Error: {error}")
            print(f"Attepting fallback")
            revised_query = search_by_text_as_fallback(previous_sys_prompt, previous_user_prompt, search_query, error)
            search_result, error = execute_query(os_client, index_name, revised_query)

            if error:
                raise Exception("Both initial and fallback queries failed. Final error: {error}")
            
        if not error:
            search_result_str = format_search_results(search_result)
            answer = generate_answer_with_text_search(search_result_str, search_query, question)
            print(f"context:\n {search_result_str}")
            print(f"answer: {answer}")
    else:
        print(f"tool not selected.")

In [ ]:
def vector_search(os_client, index_name, field, vector, weight, max_results):
    query = {
        "size": max_results,
        "_source": ["code", "title", "synopsis"],
        "query": {
            "knn": {
                field: {
                    "vector": vector,
                    "k": max_results
                }
            }
        }
    }
    try:
        response = os_client.search(index=index_name, body=query)
        return [(hit['_source'], hit['_score'] * weight) for hit in response['hits']['hits']]
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

def get_weighted_results(title_results, synopsis_results, max_results):
    combined_results = title_results + synopsis_results

    unique_results = {}
    for result, score in combined_results:
        code = result['code']  
        if code in unique_results:
            unique_results[code] = (result, max(score, unique_results[code][1]))
        else:
            unique_results[code] = (result, score)

    sorted_results = sorted(unique_results.values(), key=lambda x: x[1], reverse=True)
    return sorted_results[:max_results]


def search_by_similarity(os_client, question, max_results=10):
    model_id = "amazon.titan-embed-text-v2:0"
    question_response = boto3_client.invoke_model(
        modelId=model_id,
        body=json.dumps({"inputText": question})
    )
    question_embedding = json.loads(question_response['body'].read())['embedding']

    title_results = vector_search(os_client, index_name, "title_embedding", question_embedding, 0.4, max_results)
    synopsis_results = vector_search(os_client, index_name, "synopsis_embedding", question_embedding, 0.6, max_results)

    weighted_results = get_weighted_results(title_results, synopsis_results, max_results)
    return weighted_results
    

question = "Can you recommend sessions where I can learn about fine-tuning generative AI models?"
max_results = 10

search_result = search_by_similarity(os_client, question, max_results)
print(json.dumps(search_result, indent=2))

In [ ]:
import json

def generate_answer_with_similarity_search(search_result, question):
    sys_prompt, user_prompt = load_prompt('generate_answer_with_similarity_search')

    search_result_str = json.dumps(search_result, indent=2)

    user_prompt = user_prompt.format(
        question=question,
        search_result=search_result_str
    )

    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
    sys_prompt, user_prompt = create_prompt_without_parameter(sys_prompt, user_prompt)
    response = converse_with_bedrock(model_id, sys_prompt, user_prompt)

    return response['output']['message']['content'][0]['text']

question = "Can you recommend sessions where I can learn about fine-tuning generative AI models?"
answer = generate_answer_with_similarity_search(search_result, question)
print(answer)

In [ ]:
def augment_query_with_llm(question):
    sys_prompt, user_prompt = load_prompt('augment_query_with_llm')
    user_prompt = user_prompt.format(question=question)

    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
    sys_prompt, user_prompt = create_prompt_without_parameter(sys_prompt, user_prompt)
    response = converse_with_bedrock(model_id, sys_prompt, user_prompt)

    return response['output']['message']['content'][0]['text']

question = "Can you recommend sessions that cover best practices in DevOps?"
augmented_question = augment_query_with_llm(question)
print(augmented_question)

In [ ]:
def generate_answer_with_similarity_search_as_fallback(search_result, question, augmented_question):
    sys_prompt, user_prompt = load_prompt('generate_answer_with_similarity_search_as_fallback')

    search_result_str = json.dumps(search_result, indent=2)
    user_prompt = user_prompt.format(
        question=question,
        search_result=search_result_str
    )

    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
    sys_prompt, user_prompt = create_prompt_without_parameter(sys_prompt, user_prompt)
    response = converse_with_bedrock(model_id, sys_prompt, user_prompt)

    return response['output']['message']['content'][0]['text']

In [ ]:
for question in contextual_exploration_questions:
    response = search_tool_selection(question)
    
    print(f"\n\nquestion: {question}")
    if response == "search_by_text":
        print("\nsearch_by_text\n")
        search_query, previous_sys_prompt, previous_user_prompt = search_by_text(topics, audience_types, session_format, question)
        #print("search_query:", search_query)
        search_result, error = execute_query(os_client, index_name, search_query)
        if error:
            print(f"attempt failed. Error: {error}")
            print(f"Attepting fallback")
            revised_query = search_by_text_as_fallback(previous_sys_prompt, previous_user_prompt, search_query, error)
            search_result, error = execute_query(os_client, index_name, revised_query)

            if error:
                max_results = 10
                search_results = search_by_similarity(os_client, question, max_results)
                answer = generate_answer_with_similarity_search(search_result, question)

        if not error:
            search_result_str = format_search_results(search_result)
            answer = generate_answer_with_text_search(search_result_str, search_query, question)
            print(f"context:\n {search_result_str}")

    else:
        print("\nsearch_by_similarity\n")
        max_results = 10
        search_result = search_by_similarity(os_client, question, max_results)
        #print(json.dumps(search_result, indent=2))        
        answer = generate_answer_with_similarity_search(search_result, question)
        print("1st answer:", answer)

        if answer == "None":
            print("\nAugmenting the user query with LLM\n")
            augmented_question = augment_query_with_llm(question)
            print(f"\nAugmented question: {augmented_question}")
            search_result = search_by_similarity(os_client, augmented_question, max_results)
            print(f"\nSearch results: {search_result}")
            answer = generate_answer_with_similarity_search_as_fallback(search_result, question, augmented_question)

    print("final answer:", answer)
